# Discrete Covariance Function from time sequences from Merra2

- author Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date 2025-03-08 
- last update : 2025-03-08
- Kernel @usdf **w_2024_50*
- Office emac : mamba_py311
- Home emac : base (conda)
- laptop : conda_py311

**Goal** : Fit the variation of Merra2 parameter impact the transmission

- CO2 fit : https://scikit-learn.org/stable/auto_examples/gaussian_process/plot_gpr_co2.html#sphx-glr-auto-examples-gaussian-process-plot-gpr-co2-py

- Kernels : https://scikit-learn.org/stable/modules/gaussian_process.html#gp-kernels

In [ ]:
from platform import python_version
print(python_version())

In [ ]:
import warnings
warnings.resetwarnings()
warnings.simplefilter('ignore')

In [ ]:
from platform import python_version
print(python_version())

In [ ]:
import os

In [ ]:
# where are stored the figures
pathfigs = "figsDCTAtmosphereFomMerra2"
if not os.path.exists(pathfigs):
    os.makedirs(pathfigs)
figtype = ".png"

In [ ]:
# where are stored the figures
pathdata = "dataDCTAtmosphereFromMerra2"
if not os.path.exists(pathdata):
    os.makedirs(pathdata) 
datatype = ".csv"

dcf_path_input = os.path.join(pathdata,"dcf_timecurves") 
dcf_path_output = os.path.join(pathdata,"dcf_results") 
if not os.path.exists(dcf_path_input):
    os.makedirs(dcf_path_input) 
if not os.path.exists(dcf_path_output):
    os.makedirs(dcf_path_output) 

In [ ]:
import numpy as np
from numpy.linalg import inv
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm
from matplotlib.patches import Circle,Annulus
from astropy.visualization import ZScaleInterval
props = dict(boxstyle='round', facecolor="white", alpha=0.1)
#props = dict(boxstyle='round')

import matplotlib.colors as colors
import matplotlib.cm as cmx

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from matplotlib.gridspec import GridSpec

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits
from astropy.wcs import WCS
from astropy import units as u
from astropy import constants as c

from astropy.coordinates.earth import EarthLocation
from datetime import datetime
from pytz import timezone

from scipy import interpolate
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KDTree, BallTree

import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 100)

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
import pickle
from collections import OrderedDict

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

import scipy
from scipy.optimize import curve_fit,least_squares

In [ ]:
props = dict(boxstyle='round', facecolor='white', alpha=0.5)

In [ ]:
from astropy.modeling import models

In [ ]:
from numpy.random import lognormal

In [ ]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time
from astropy.timeseries import TimeSeries

from astropy.modeling import models, fitting

In [ ]:
# Remove to run faster the notebook
import ipywidgets as widgets
%matplotlib widget

In [ ]:
from importlib.metadata import version

In [ ]:
# wavelength bin colors
#jet = plt.get_cmap('jet')
#cNorm = mpl.colors.Normalize(vmin=0, vmax=NSED)
#scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
#all_colors = scalarMap.to_rgba(np.arange(NSED), alpha=1)

In [ ]:
np.__version__

In [ ]:
pd.__version__

In [ ]:
from scipy.fftpack import fft, fftfreq

from pyzdcf import pyzdcf

In [ ]:
def convertNumToDatestr(num):
    year = num//10_000
    month= (num-year*10_000)//100
    day = (num-year*10_000-month*100)

    year_str = str(year).zfill(4)
    month_str = str(month).zfill(2)
    day_str = str(day).zfill(2)
    
    datestr = f"{year_str}-{month_str}-{day_str}"
    return pd.to_datetime(datestr)

In [ ]:
def pdf_lognormal(x,a0,mu,sigma):
    """
    """
    pdf = a0*(np.exp(-(np.log(x) - mu)**2 / (2 * sigma**2))/ (x * sigma * np.sqrt(2 * np.pi)))
    return pdf

In [ ]:
FIGXSIZE_1 = 14
FIGYSIZE_1 = 8

FIGXSIZE_0 = 14
FIGYSIZE_0 = 7

In [ ]:
YEAR = 365.25
MONTHS6 = YEAR/2.
MONTHS4 = YEAR/3.
QUARTER = YEAR/4. 
DAY = 1.
MONTH = YEAR/12.
WEEK = 7*DAY

In [ ]:
def fourier_analysis_old(dates, values,mode = "logxlogy",title="Analyse de Fourier - Spectre des fréquences",
                    xlabel="Fréquence (cycles par jour)",ylabel="Amplitude",label="Amplitude spectrale", legendout = True, datecut = 59500,figname=""):
    # Centrer les données autour de la moyenne


    if datecut>0:
        index_selected = np.where(dates >= datecut)[0]
        dates = dates[index_selected]
        values = values[index_selected]

    
    values_centered = values - np.mean(values)
    

    # Nombre de points
    N = len(dates)

    # sigma
    sigma_x = np.sqrt(np.sum(values_centered**2)/N)
    
    # Intervalle d'échantillonnage (assume 1 jour entre chaque point)
    T = np.mean(np.diff(dates))  # Période d'échantillonnage

    # Fréquence de Nyquist (limite de Shannon)
    f_nyquist = 1 / (2 * T)
    
    # Transformée de Fourier
    fft_values = fft(values_centered)/ np.sqrt(N)
    freqs = fftfreq(N, T)  # Fréquences associées

    

    # Seulement la moitié du spectre est utile (symétrie)
    positive_freqs = freqs[:N // 2]
    positive_fft_values = np.abs(fft_values[:N // 2])

    # Tracer le spectre
    fig, ax = plt.subplots(1,1,figsize=(FIGXSIZE_0, FIGYSIZE_0),layout="constrained")
    
    ax.plot(positive_freqs, positive_fft_values,'ob-' ,ms=5,label=label)

    if mode == "logxliny":
        plt.xscale("log")  # Définit l'axe X en échelle logarithmique
        plt.yscale("linear")  # Garde l'axe Y en échelle linéaire
    elif mode == "logxlogy":
        plt.xscale("log")  # Définit l'axe X en échelle logarithmique
        plt.yscale("log")  # Garde l'axe Y en échelle logarithmique
    elif mode == "linxlogy":
        plt.xscale("linear")  # Définit l'axe X en  échelle linéaire
        plt.yscale("log")  # Garde l'axe Y en échelle logarithmique
    elif mode == "linxliny":
        plt.xscale("linear")  # Définit l'axe X en échelle linéaire
        plt.yscale("linear")  # Garde l'axe Y en échelle linéaire
        
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    

    ax.axvline(1/YEAR, color='r', linestyle='-', label="Cycle : 365 days - 1 year")
    ax.axvline(1/MONTHS6, color='r', linestyle='--', label="Cycle : 182.6 days - 6 months")
    ax.axvline(1/MONTHS4, color='r', linestyle=':', label="Cycle : 121.7 days - 4 months")
    ax.axvline(1/QUARTER, color='r', linestyle='-.', label="Cycle : 91.3 days - 3 months")
    ax.axvline(1/MONTH, color='r', linestyle=':', label="Cycle : 30.4 days - 1 month")
    ax.axvline(1/WEEK, color='purple', linestyle='--', label="Cycle : 7 days - 1 week")
    ax.axvline(DAY, color='purple', linestyle='-', label="Cycle : 1 day ")
    ax.axvline(1./(0.5*DAY), color='purple', linestyle='-.', label="Cycle : 0.5 day ")

    ax.axvline(f_nyquist, color='g', linestyle='--', label=f"Nyquist frequency({f_nyquist:.3f} cycles/days)")

    txtstr_sigma = "$\sigma_x$ = " + f" {sigma_x:0.3f}" 
    ax.text(0.01, 0.95, txtstr_sigma, transform=ax.transAxes, fontsize=16,verticalalignment='top', bbox=props)
    

    if legendout:
        ax.legend(bbox_to_anchor=(1.05, 1.05),fontsize=12)
    else:
        ax.legend(fontsize=10,fancybox=True, framealpha=0.5)
        
    

    if figname !="":
        fig.savefig(figname)
        
    
    plt.show()

# Appelle la fonction avec tes données
# fourier_analysis(dates, values)

In [ ]:
def fourier_analysis(dates, values, ax, mode = "logxlogy",title="Analyse de Fourier - Spectre des fréquences",
                    xlabel="frequency (day)$^{-1}$",ylabel="y-unit $\cdot \sqrt{day}$",label="DFT amplitude", legendout = True, datecut = 59500):
    # Centrer les données autour de la moyenne


    if datecut>0:
        index_selected = np.where(dates >= datecut)[0]
        dates = dates[index_selected]
        values = values[index_selected]
    
    values_centered = values - np.mean(values)

    # Nombre de points
    N = len(dates)
    # Intervalle d'échantillonnage (assume 1 jour entre chaque point)
    T = np.mean(np.diff(dates))  # Période d'échantillonnage

    # signal duration
    Delta_T = dates.max()-dates.min()

    #frequency spacing
    Delta_f = 1/Delta_T
    

    # Fréquence de Nyquist (limite de Shannon)
    f_nyquist = 1 / (2 * T)
    
    # Transformée de Fourier
    fft_values = fft(values_centered)/ np.sqrt(N)
    freqs = fftfreq(N, T)  # Fréquences associées

    # sigma_x
    sigma_x = np.sqrt(np.sum(values_centered**2)/N)

    

    # Seulement la moitié du spectre est utile (symétrie)
    positive_freqs = freqs[:N // 2]
    positive_fft_values = np.abs(fft_values[:N // 2])

    # Calcul de l'écart-type fréquentiel en Hz
    power_spectrum = np.abs(positive_fft_values) ** 2
    sigma_f = np.sqrt(np.sum(positive_freqs**2 * power_spectrum) / np.sum(power_spectrum))
    
    sigma_pp = np.sqrt(np.sum(power_spectrum))
    sigma_x = np.sqrt(np.sum(values_centered**2)/N)

    # Tracer le spectre
    #plt.figure(figsize=(16, 6),layout="constrained")
    
    ax.plot(positive_freqs, positive_fft_values,'ob-' ,ms=5,label=label)

    if mode == "logxliny":
        ax.set_xscale("log")  # Définit l'axe X en échelle logarithmique
        ax.set_yscale("linear")  # Garde l'axe Y en échelle linéaire
    elif mode == "logxlogy":
        ax.set_xscale("log")  # Définit l'axe X en échelle logarithmique
        ax.set_yscale("log")  # Garde l'axe Y en échelle logarithmique
    elif mode == "linxlogy":
        ax.set_xscale("linear")  # Définit l'axe X en  échelle linéaire
        ax.set_yscale("log")  # Garde l'axe Y en échelle logarithmique
    elif mode == "linxliny":
        ax.set_xscale("linear")  # Définit l'axe X en échelle linéaire
        ax.set_yscale("linear")  # Garde l'axe Y en échelle linéaire
        
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    

    ax.axvline(1/YEAR, color='r', linestyle='-', label="Cycle : 365 days - 1 year")
    ax.axvline(1/MONTHS6, color='r', linestyle='--', label="Cycle : 182.6 days - 6 months")
    ax.axvline(1/MONTHS4, color='r', linestyle=':', label="Cycle : 121.7 days - 4 months")
    ax.axvline(1/QUARTER, color='r', linestyle='-.', label="Cycle : 91.3 days - 3 months")
    ax.axvline(1/MONTH, color='r', linestyle=':', label="Cycle : 30.4 days - 1 month")
    ax.axvline(1/WEEK, color='purple', linestyle='--', label="Cycle : 7 days - 1 week")
    ax.axvline(DAY, color='purple', linestyle='-', label="Cycle : 1 day ")
    ax.axvline(1./(0.5*DAY), color='purple', linestyle='-.', label="Cycle : 0.5 day ")

    ax.axvline(f_nyquist, color='g', linestyle='--', label=f"Nyquist frequency({f_nyquist:.3f} cycles/days)")

    if legendout:
        ax.legend(bbox_to_anchor=(1.05, 1.05),fontsize=12)
    else:
        ax.legend(fontsize=10,fancybox=True, framealpha=0.5)

    txtstr_sigma = "$\sigma_x$ = " + f" {sigma_x:0.3f}" 
    txtstr_Ts = "$T_s$ = "  + f" {T:0.3f} days" 
    txtstr_Fs = "$f_s$ = "  + f" {Delta_f:0.3f} / days" 

    txtstr_info = "\n".join([txtstr_sigma,txtstr_Ts ,txtstr_Fs])
    ax.text(0.01, 0.95, txtstr_info, transform=ax.transAxes, fontsize=16,verticalalignment='top', bbox=props)
    
    
 
    
# Appelle la fonction avec tes données
# fourier_analysis(dates, values)
    return positive_freqs, positive_fft_values, sigma_x,sigma_pp

In [ ]:
def ComputeZDCF(filename_in,df_pwv_curve,minpts=20):
    """
    Compute the Discrete Covariance Curve with pyzdcf

    parameters :
    - df_pwv_curve : pandas dataframe with 3 columns : (time, pwv, sigma)
      The time has to be chosen in terms of days/hours,min ..., outside this function
    - filename_in : csv file where are written the tempory DTC curve
    
    """
    


    # add the error on the point 
    #df_pwvc = df_pwvc.assign(sig_pwv = lambda x: sigma_repeatability)

    full_filename_in = os.path.join(dcf_path_input,filename_in)
    df_pwv_curve.to_csv(full_filename_in, index=False,header=False)

    # parameters for the pyzdcf
    params_dcf = dict(autocf    =  True, # Autocorrelation (T) or cross-correlation (F)
              prefix            = 'acf',  # Output files prefix
              uniform_sampling  =  False, # Uniform sampling?
              omit_zero_lags    =  False,  # Omit zero lag points?
              minpts            =  minpts,     # Min. num. of points per bin (0 is a flag for default value of 11)
              num_MC            =  100,   # Num. of Monte Carlo simulations for error estimation
              lc1_name          =  filename_in,   # Name of the first light curve file
              lc2_name          =  filename_in    # Name of the second light curve file (required only if we do CCF)
             )

    # compute the ZDCF
    
    dcf_df = pyzdcf(input_dir  =  dcf_path_input + "/" , 
                    output_dir = dcf_path_output + "/", 
                    intr       = False, 
                    parameters = params_dcf, 
                    sep        = ',', 
                    sparse     = 'auto', 
                    verbose    = False)
    return dcf_df

## Configuration

In [ ]:
tmin_select = 59500

In [ ]:
observing_location = EarthLocation.of_site('Rubin Observatory')
tz = timezone('America/Santiago')

### MERRA2

In [ ]:
filename_m2 = "../../SpectroMerra2/MerradataMerged/Merge_inst1_2d_asm_Nx_M2I1NXASM-2021-2024.csv"
filename_m2b = "../../SpectroMerra2/MerradataMerged/Merge_tavg1_2d_aer_Nx_M2T1NXAER-2021-2024.csv"

In [ ]:
df_m = pd.read_csv(filename_m2)
df_mb = pd.read_csv(filename_m2b)

In [ ]:
Nm = len(df_m)
Nmb = len(df_mb)
print("Number of points :: ",Nm,Nmb)

In [ ]:
df_mb.columns

In [ ]:
TMIN = pd.to_datetime(df_m.time.min())
TMAX = pd.to_datetime(df_m.time.max())

### Convert in MJD

In [ ]:
df_m["mjd"] = Time(pd.to_datetime(df_m.time.values)).mjd
df_mb["mjd"] = Time(pd.to_datetime(df_mb.time.values)).mjd

In [ ]:
mjd_zoom_start = Time("2024-01-01").mjd
mjd_zoom_stop = Time("2025-06-30").mjd

In [ ]:
mjd_obs_start = df_m["mjd"].min() 
mjd_obs_stop = df_m["mjd"].max() 

## PWV

In [ ]:
from matplotlib.dates import DateFormatter
#date_form = DateFormatter("%y-%m-%dT%H:%M")
date_form = DateFormatter("%y-%m")

fig = plt.figure(figsize=(18,10))
gs = GridSpec(2, 1,figure=fig)
#gs = GridSpec(1, 1,figure=fig)
ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1])
        
leg1=ax1.get_legend()
leg2=ax2.get_legend()


ax1.plot(pd.to_datetime(df_m.Time.values), df_m.TQV.values,c="b",lw=0.5,label="Merra2")
ax1.set_xlabel("time")
ax1.xaxis.set_major_formatter(date_form)
ax1.set_title("Precipitable water vapor from Merra2")
ax1.legend()
ax1.set_ylabel("PWV (mm)")
#ax.set_xlim(TMIN,TMAX)

data = df_m.TQV.values
mean = np.mean(data)
median = np.median(data)
std = np.std(data)
textstr = "\n".join((f"Expected max-range for PWV : ",
                     f"- average : {mean:.2f} mm",
                     f"- median : {median:.2f} mm",
                     f"- sigma : {std:.2f} mm",     
                    ))
ax1.text(0.05, 0.95, textstr, transform=ax1.transAxes, fontsize=14,verticalalignment='top', bbox=props)


ax2.plot(df_m.mjd, df_m.TQV.values,c="b",lw=0.5,label="Merra2")
ax2.set_xlabel("time (MJD)")
ax2.legend()
ax2.set_ylabel("PWV (mm)")

figname =f"{pathfigs}/pwv_allpoints_merra2"+figtype
fig.savefig(figname)
plt.show()


### DCT for PWV

In [ ]:
df_m = df_m[df_m.mjd>tmin_select]
Nm = len(df_m)

In [ ]:
NSAMP = 10000
index_range = np.arange(0,Nm ,1)
index_selected = np.random.choice(index_range , size=NSAMP,replace = False)
index_selected = np.sort(index_selected)
index_selected = index_range 

In [ ]:
SIGMA_PWVREPEATABILITY = 0.26

In [ ]:
df_dcf_in = df_m[["mjd","TQV"]]
tstart = df_dcf_in["mjd"].min()
tstop = df_dcf_in["mjd"].max()
df_dcf_in["t_day"] = df_dcf_in["mjd"] - tstart
df_dcf_in = df_dcf_in[["t_day","TQV"]] 

In [ ]:
df_dcf_in = df_dcf_in.assign(sig_pwv = lambda x: SIGMA_PWVREPEATABILITY)
df_dcf_in = df_dcf_in.iloc[index_selected]

In [ ]:
df_dcf_out = ComputeZDCF("dcf_in_pwv_merra2.csv",df_dcf_in, minpts=20)

In [ ]:
xerr = df_dcf_out[["-sig(tau)","+sig(tau)"]].values.T	
yerr = df_dcf_out[["-err(dcf)","+err(dcf)"]].values.T	
x = df_dcf_out["tau"].values
y = df_dcf_out["dcf"].values

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(12,5),layout="constrained")
ax.errorbar(x,y,xerr=xerr,yerr=yerr,marker='o', mfc='red',linewidth=0.5,
         mec='blue', ms=2, mew=2,ecolor="k",elinewidth=2,capsize=2,uplims=True, lolims=True)
ax.grid()
ax.set_ylim(-1,1)
ax.set_title(f"Discrete covariance function on PWV measurements in Merra2")
ax.set_xlabel("Time (days)")
ax.set_ylabel("DCF (no units)")

## Ozone

In [ ]:
from matplotlib.dates import DateFormatter
#date_form = DateFormatter("%y-%m-%dT%H:%M")
date_form = DateFormatter("%y-%m")

fig = plt.figure(figsize=(18,10))
gs = GridSpec(2, 1,figure=fig)

ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1])
        
leg1=ax1.get_legend()
leg2=ax2.get_legend()


ax1.plot(pd.to_datetime(df_m.Time.values), df_m.TO3.values,c="r",lw=0.5,label="Merra2")
ax1.set_xlabel("time")
ax1.xaxis.set_major_formatter(date_form)
ax1.set_title("Ozone from Merra2")
#ax1.legend()
ax1.set_ylabel("Ozone (DU)")
#ax.set_xlim(TMIN,TMAX)

data = df_m.TO3.values
mean = np.mean(data)
median = np.median(data)
std = np.std(data)
textstr = "\n".join((f"Expected range for Ozone : ",
                     f"- average : {mean:.2f} DU",
                     f"- median : {median:.2f} DU",
                     f"- sigma : {std:.2f} DU",     
                    ))
ax1.text(0.05, 0.95, textstr, transform=ax1.transAxes, fontsize=14,verticalalignment='top', bbox=props)

ax2.plot(df_m.mjd, df_m.TO3.values,c="r",lw=0.5,label="Merra2")
ax2.set_xlabel("time (MJD)")
ax2.legend()
ax2.set_ylabel("Ozone (DU)")


figname =f"{pathfigs}/ozone_allpoints_merra2"+figtype
fig.savefig(figname)
plt.show()


### DCT for ozone

In [ ]:
SIGMA_OZONEREPEATABILITY = 5.

In [ ]:
df_m = df_m[df_m.mjd>tmin_select]
Nm = len(df_m)

In [ ]:
NSAMP = 10000
index_range = np.arange(0,Nm ,1)
index_selected = np.random.choice(index_range , size=NSAMP,replace = False)
index_selected = np.sort(index_selected)
index_selected = index_range

In [ ]:
df_dcf_in = df_m[["mjd","TO3"]]
tstart = df_dcf_in["mjd"].min()
tstop = df_dcf_in["mjd"].max()
df_dcf_in["t_day"] = df_dcf_in["mjd"] - tstart
df_dcf_in = df_dcf_in[["t_day","TO3"]] 

In [ ]:
df_dcf_in = df_dcf_in.assign(sig_ozone = lambda x: SIGMA_OZONEREPEATABILITY )
df_dcf_in = df_dcf_in.iloc[index_selected]

In [ ]:
df_dcf_out = ComputeZDCF("dcf_in_ozone_merra2.csv",df_dcf_in, minpts = 20)

In [ ]:
xerr = df_dcf_out[["-sig(tau)","+sig(tau)"]].values.T	
yerr = df_dcf_out[["-err(dcf)","+err(dcf)"]].values.T	
x = df_dcf_out["tau"].values
y = df_dcf_out["dcf"].values

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(12,5),layout="constrained")
ax.errorbar(x,y,xerr=xerr,yerr=yerr,marker='o', mfc='red',linewidth=0.5,
         mec='red', ms=2, mew=2,ecolor="k",elinewidth=2,capsize=2,uplims=True, lolims=True)
ax.grid()
ax.set_ylim(-1,1)
ax.set_title(f"Discrete covariance function on Ozone measurements in Merra2")
ax.set_xlabel("Time (days)")
ax.set_ylabel("DCF (no units)")

## Aerosol VAOD

In [ ]:
from matplotlib.dates import DateFormatter
#date_form = DateFormatter("%y-%m-%dT%H:%M")
date_form = DateFormatter("%y-%m")

fig = plt.figure(figsize=(18,10))
gs = GridSpec(2, 1,figure=fig)
ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1])
        
leg1=ax1.get_legend()
leg2=ax2.get_legend()


ax1.plot(pd.to_datetime(df_mb.Time.values), df_mb.TOTEXTTAU.values,c="g",lw=0.5,label="Merra2")
ax1.set_xlabel("time")
ax1.xaxis.set_major_formatter(date_form)
ax1.set_title("VAOD from Merra2")
ax1.legend()
ax1.set_ylabel("VAOD")
#ax.set_xlim(TMIN,TMAX)
data = df_mb.TOTEXTTAU.values
mean = np.mean(data)
median = np.median(data)
std = np.std(data)
textstr = "\n".join((f"Expected max-range for VAOD : ",
                     f"- average : {mean:.3f}",
                     f"- median : {median:.3f}",
                     f"- sigma : {std:.3f}",     
                    ))
ax1.text(0.05, 0.95, textstr, transform=ax1.transAxes, fontsize=14,verticalalignment='top', bbox=props)

ax2.plot(df_mb.mjd, df_mb.TOTEXTTAU.values,c="g",lw=0.5,label="Merra2")
ax2.set_xlabel("time (MJD)")
ax2.legend()
ax2.set_ylabel("VAOD")


figname =f"{pathfigs}/vaod_allpoints_merra2"+figtype
fig.savefig(figname)
plt.show()


### DCT for VAOD 

In [ ]:
SIGMA_VAODREPEATABILITY = 0.01

In [ ]:
df_mb = df_mb[df_mb.mjd>tmin_select]
Nmb = len(df_mb)

In [ ]:
NSAMP = 10000
index_range = np.arange(0,Nmb ,1)
index_selected = np.random.choice(index_range , size=NSAMP,replace = False)
index_selected = np.sort(index_selected)
index_selected = index_range

In [ ]:
df_dcf_in = df_mb[["mjd","TOTEXTTAU"]]
tstart = df_dcf_in["mjd"].min()
tstop = df_dcf_in["mjd"].max()
df_dcf_in["t_day"] = df_dcf_in["mjd"] - tstart
df_dcf_in = df_dcf_in[["t_day","TOTEXTTAU"]] 

In [ ]:
df_dcf_in = df_dcf_in.assign(sig_vaod = lambda x: SIGMA_VAODREPEATABILITY )
df_dcf_in = df_dcf_in.iloc[index_selected]

In [ ]:
df_dcf_out = ComputeZDCF("dcf_in_vaod_merra2.csv",df_dcf_in, minpts = 20 )

In [ ]:
xerr = df_dcf_out[["-sig(tau)","+sig(tau)"]].values.T	
yerr = df_dcf_out[["-err(dcf)","+err(dcf)"]].values.T	
x = df_dcf_out["tau"].values
y = df_dcf_out["dcf"].values

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(12,5),layout="constrained")
ax.errorbar(x,y,xerr=xerr,yerr=yerr,marker='o', mfc='green',linewidth=0.5,
         mec='green', ms=2, mew=2,ecolor="k",elinewidth=2,capsize=2,uplims=True, lolims=True)
ax.grid()
ax.set_ylim(-1,1)
ax.set_title(f"Discrete covariance function on VAOD measurements in Merra2")
ax.set_xlabel("Time (days)")
ax.set_ylabel("DCF (no units)")

## Aerosol Angstrom

In [ ]:
from matplotlib.dates import DateFormatter
#date_form = DateFormatter("%y-%m-%dT%H:%M")
date_form = DateFormatter("%y-%m")

fig = plt.figure(figsize=(18,10))
gs = GridSpec(2, 1,figure=fig)

ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1])
        
leg1=ax1.get_legend()
leg2=ax2.get_legend()


ax1.plot(pd.to_datetime(df_mb.Time.values), df_mb.TOTANGSTR.values,c="purple",lw=0.5,label="Merra2")
ax1.set_xlabel("time")
ax1.xaxis.set_major_formatter(date_form)
ax1.set_title("VAOD_Angstrom from Merra2")
ax1.legend()
ax1.set_ylabel("VAOD_Angstrom")
#ax.set_xlim(TMIN,TMAX)
data = df_mb.TOTANGSTR.values
mean = np.mean(data)
median = np.median(data)
std = np.std(data)
textstr = "\n".join((f"Expected max-range for VAOD-Angstrom : ",
                     f"- average : {mean:.3f}",
                     f"- median : {median:.3f}",
                     f"- sigma : {std:.3f}",     
                    ))
ax1.text(0.05, 0.95, textstr, transform=ax1.transAxes, fontsize=14,verticalalignment='top', bbox=props)


ax2.plot(df_mb.mjd, df_mb.TOTANGSTR.values,c="purple",lw=0.5,label="Merra2")
ax2.set_xlabel("time (MJD)")
ax2.legend()
ax2.set_ylabel("VAOD")



figname =f"{pathfigs}/vaodangstrom_allpoints_merra2"+figtype
fig.savefig(figname)
plt.show()

### DCT for Angstrom

In [ ]:
SIGMA_ANGSTROMREPEATABILITY = 0.008

In [ ]:
df_mb = df_mb[df_mb.mjd>tmin_select]
Nmb = len(df_mb)

In [ ]:
NSAMP = 10000
index_range = np.arange(0,Nmb ,1)
index_selected = np.random.choice(index_range , size=NSAMP,replace = False)
index_selected = np.sort(index_selected)
index_selected = index_range

In [ ]:
df_dcf_in = df_mb[["mjd","TOTANGSTR"]]
tstart = df_dcf_in["mjd"].min()
tstop = df_dcf_in["mjd"].max()
df_dcf_in["t_day"] = df_dcf_in["mjd"] - tstart
df_dcf_in = df_dcf_in[["t_day","TOTANGSTR"]] 

In [ ]:
df_dcf_in = df_dcf_in.assign(sig_ozone = lambda x: SIGMA_ANGSTROMREPEATABILITY )
df_dcf_in = df_dcf_in.iloc[index_selected]

In [ ]:
df_dcf_out = ComputeZDCF("dcf_in_angstrom_merra2.csv",df_dcf_in, minpts = 20)

In [ ]:
xerr = df_dcf_out[["-sig(tau)","+sig(tau)"]].values.T	
yerr = df_dcf_out[["-err(dcf)","+err(dcf)"]].values.T	
x = df_dcf_out["tau"].values
y = df_dcf_out["dcf"].values

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(12,5),layout="constrained")
ax.errorbar(x,y,xerr=xerr,yerr=yerr,marker='o', mfc='red',linewidth=0.5,
         mec='purple', ms=2, mew=2,ecolor="k",elinewidth=2,capsize=2,uplims=True, lolims=True)
ax.grid()
ax.set_ylim(-1,1)
ax.set_title(f"Discrete covariance function on Angstrom exponent measurements in Merra2")
ax.set_xlabel("Time (days)")
ax.set_ylabel("DCF (no units)")